# 📊 Dogłębna Analiza Zadań Kombinatorycznych

Analiza pliku: `src/data/kombinatoryka-problems.json`

In [ ]:
import json
import pandas as pd
import numpy as np
from collections import Counter, defaultdict
import matplotlib.pyplot as plt
import seaborn as sns

# Ustawienia wyświetlania
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## 1. Wczytanie Danych

In [ ]:
# Wczytanie danych
with open('src/data/kombinatoryka-problems.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

print(f"🔢 Liczba zadań: {len(data)}")
print(f"📝 Rozmiar pliku: {len(json.dumps(data))} znaków")

## 2. Podstawowe Statystyki

In [ ]:
# Utworzenie DataFrame z podstawowymi informacjami
basic_info = []
for p in data:
    basic_info.append({
        'id': p.get('id', 'N/A'),
        'topic': p.get('topic', 'N/A'),
        'difficulty': p.get('difficulty', 0),
        'num_steps': len(p.get('steps', [])),
        'has_solution': len(p.get('solutions', [])) > 0,
        'num_solutions': len(p.get('solutions', [])),
        'timestamp': p.get('timestamp', 'N/A')
    })

df_basic = pd.DataFrame(basic_info)
print("\n📊 Podstawowe informacje o zadaniach:")
df_basic.head(10)

In [ ]:
# Statystyki opisowe
print("\n📈 Statystyki opisowe:")
df_basic[['difficulty', 'num_steps', 'num_solutions']].describe()

## 3. Analiza Poziomów Trudności

In [ ]:
# Rozkład poziomów trudności
difficulty_counts = df_basic['difficulty'].value_counts().sort_index()

print("\n🎯 Rozkład poziomów trudności:")
for diff, count in difficulty_counts.items():
    percentage = (count / len(df_basic)) * 100
    print(f"  Poziom {diff}: {count} zadań ({percentage:.1f}%)")

# Wykres
fig, ax = plt.subplots(1, 2, figsize=(14, 5))

# Wykres słupkowy
difficulty_counts.plot(kind='bar', ax=ax[0], color='steelblue')
ax[0].set_title('Rozkład Poziomów Trudności', fontsize=14, fontweight='bold')
ax[0].set_xlabel('Poziom trudności')
ax[0].set_ylabel('Liczba zadań')
ax[0].grid(axis='y', alpha=0.3)

# Wykres kołowy
difficulty_counts.plot(kind='pie', ax=ax[1], autopct='%1.1f%%', startangle=90)
ax[1].set_title('Procentowy Rozkład Trudności', fontsize=14, fontweight='bold')
ax[1].set_ylabel('')

plt.tight_layout()
plt.show()

## 4. Analiza Kroków (Steps)

In [ ]:
# Korelacja trudność vs liczba kroków
print("\n📚 Korelacja: Trudność vs Liczba Kroków")
difficulty_steps_stats = df_basic.groupby('difficulty')['num_steps'].agg(['mean', 'min', 'max', 'std'])
print(difficulty_steps_stats)

# Wykres pudełkowy
plt.figure(figsize=(10, 6))
df_basic.boxplot(column='num_steps', by='difficulty', grid=False)
plt.suptitle('')
plt.title('Rozkład Liczby Kroków według Poziomu Trudności', fontsize=14, fontweight='bold')
plt.xlabel('Poziom trudności')
plt.ylabel('Liczba kroków')
plt.show()

In [ ]:
# Szczegółowa analiza kroków
step_analysis = []

for p in data:
    for step in p.get('steps', []):
        step_analysis.append({
            'problem_id': p.get('id'),
            'has_hint': 'hint' in step,
            'has_expression': 'expression' in step,
            'has_explanation': 'explanation' in step,
            'has_why': 'why' in step,
            'has_interactive_choice': 'interactive_choice' in step
        })

df_steps = pd.DataFrame(step_analysis)

print("\n🔍 Analiza kompletności kroków:")
print(f"  Całkowita liczba kroków: {len(df_steps)}")
print(f"  Kroków z 'hint': {df_steps['has_hint'].sum()} ({100*df_steps['has_hint'].mean():.1f}%)")
print(f"  Kroków z 'expression': {df_steps['has_expression'].sum()} ({100*df_steps['has_expression'].mean():.1f}%)")
print(f"  Kroków z 'explanation': {df_steps['has_explanation'].sum()} ({100*df_steps['has_explanation'].mean():.1f}%)")
print(f"  Kroków z 'why': {df_steps['has_why'].sum()} ({100*df_steps['has_why'].mean():.1f}%)")
print(f"  Kroków z 'interactive_choice': {df_steps['has_interactive_choice'].sum()} ({100*df_steps['has_interactive_choice'].mean():.1f}%)")

## 5. Analiza Interactive Choices

In [ ]:
# Szczegółowa analiza pytań interaktywnych
ic_analysis = []

for p in data:
    for step in p.get('steps', []):
        if 'interactive_choice' in step:
            ic = step['interactive_choice']
            options = ic.get('options', [])
            
            correct_count = sum(1 for opt in options if opt.get('is_correct', False))
            all_have_feedback = all('feedback' in opt for opt in options)
            
            ic_analysis.append({
                'problem_id': p.get('id'),
                'num_options': len(options),
                'num_correct': correct_count,
                'all_have_feedback': all_have_feedback,
                'has_prompt': 'prompt' in ic,
                'has_explanation_after': 'explanation_after' in ic
            })

if ic_analysis:
    df_ic = pd.DataFrame(ic_analysis)
    
    print("\n🎮 Analiza Interactive Choices:")
    print(f"  Liczba pytań interaktywnych: {len(df_ic)}")
    print(f"  Średnia liczba opcji: {df_ic['num_options'].mean():.2f}")
    print(f"  Pytań gdzie wszystkie opcje mają feedback: {df_ic['all_have_feedback'].sum()} ({100*df_ic['all_have_feedback'].mean():.1f}%)")
    print(f"  Pytań z 'explanation_after': {df_ic['has_explanation_after'].sum()} ({100*df_ic['has_explanation_after'].mean():.1f}%)")
    
    # Rozkład liczby opcji
    print("\n  Rozkład liczby opcji:")
    print(df_ic['num_options'].value_counts().sort_index())
    
    # Rozkład liczby poprawnych odpowiedzi
    print("\n  Rozkład liczby poprawnych odpowiedzi:")
    print(df_ic['num_correct'].value_counts().sort_index())
else:
    print("\n⚠️  Brak pytań interaktywnych")

## 6. Analiza Tematyczna

In [ ]:
# Rozkład tematów
topic_counts = df_basic['topic'].value_counts()

print("\n📚 Top 15 Tematów:")
print(topic_counts.head(15))

# Wykres
plt.figure(figsize=(14, 6))
topic_counts.head(15).plot(kind='barh', color='coral')
plt.title('Top 15 Najczęstszych Tematów', fontsize=14, fontweight='bold')
plt.xlabel('Liczba zadań')
plt.ylabel('Temat')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

## 7. Analiza Rozwiązań

In [ ]:
# Analiza rozwiązań
import re

solution_analysis = {
    'no_solution': 0,
    'single_solution': 0,
    'multiple_solutions': 0,
    'integer_solutions': 0,
    'fraction_solutions': 0,
    'formula_solutions': 0
}

for p in data:
    solutions = p.get('solutions', [])
    
    if len(solutions) == 0:
        solution_analysis['no_solution'] += 1
    elif len(solutions) == 1:
        solution_analysis['single_solution'] += 1
    else:
        solution_analysis['multiple_solutions'] += 1
    
    # Analiza typów rozwiązań
    for sol in solutions:
        if re.search(r'\\frac', sol):
            solution_analysis['fraction_solutions'] += 1
        elif re.search(r'^\$\d+\$$', sol) or re.search(r'^\$\d+\\,\d+\$$', sol):
            solution_analysis['integer_solutions'] += 1
        else:
            solution_analysis['formula_solutions'] += 1

print("\n💡 Analiza Rozwiązań:")
for key, value in solution_analysis.items():
    print(f"  {key}: {value}")

## 8. Analiza Konceptów Matematycznych

In [ ]:
# Wyszukiwanie kluczowych konceptów
concepts = {
    'permutacja': 0,
    'wariacja': 0,
    'kombinacja': 0,
    'silnia': 0,
    'dwumian': 0,
    'zasada mnożenia': 0,
    'zasada dodawania': 0,
    'zasada włączeń': 0,
    'podzielność': 0,
    'ciąg': 0,
    'NWW': 0,
    'NWD': 0
}

for p in data:
    # Konwertujemy całe zadanie na tekst
    text = json.dumps(p, ensure_ascii=False).lower()
    
    for concept in concepts:
        if concept.lower() in text:
            concepts[concept] += 1

# Sortowanie i wyświetlanie
concepts_sorted = sorted(concepts.items(), key=lambda x: x[1], reverse=True)

print("\n🧮 Częstość występowania konceptów matematycznych:")
for concept, count in concepts_sorted:
    if count > 0:
        percentage = (count / len(data)) * 100
        print(f"  {concept}: {count} zadań ({percentage:.1f}%)")

# Wykres
concepts_df = pd.DataFrame(concepts_sorted, columns=['Koncept', 'Liczba'])
concepts_df = concepts_df[concepts_df['Liczba'] > 0]

plt.figure(figsize=(12, 6))
plt.barh(concepts_df['Koncept'], concepts_df['Liczba'], color='teal')
plt.title('Częstość Występowania Konceptów Matematycznych', fontsize=14, fontweight='bold')
plt.xlabel('Liczba zadań')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

## 9. Analiza Walidacji

In [ ]:
# Sprawdzenie statusu walidacji
validation_stats = {
    'has_validation': 0,
    'is_valid': 0,
    'has_warnings': 0,
    'has_missing_fields': 0
}

warnings_list = []
missing_fields_list = []

for p in data:
    if 'validation' in p:
        validation_stats['has_validation'] += 1
        
        val = p['validation']
        if val.get('isValid', False):
            validation_stats['is_valid'] += 1
        
        if val.get('warnings', []):
            validation_stats['has_warnings'] += 1
            warnings_list.append((p.get('id'), val['warnings']))
        
        if val.get('missingFields', []):
            validation_stats['has_missing_fields'] += 1
            missing_fields_list.append((p.get('id'), val['missingFields']))

print("\n✅ Statystyki Walidacji:")
print(f"  Zadania z sekcją validation: {validation_stats['has_validation']}/{len(data)}")
print(f"  Zadania z isValid=true: {validation_stats['is_valid']}/{len(data)}")
print(f"  Zadania z ostrzeżeniami: {validation_stats['has_warnings']}")
print(f"  Zadania z brakującymi polami: {validation_stats['has_missing_fields']}")

if warnings_list:
    print("\n⚠️  Ostrzeżenia (pierwsze 5):")
    for pid, warns in warnings_list[:5]:
        print(f"  {pid}: {warns}")

if missing_fields_list:
    print("\n❌ Brakujące pola (pierwsze 5):")
    for pid, missing in missing_fields_list[:5]:
        print(f"  {pid}: {missing}")

## 10. Analiza Duplikatów

In [ ]:
# Sprawdzenie duplikatów ID
id_counts = Counter([p.get('id') for p in data])
duplicates = [(id_val, count) for id_val, count in id_counts.items() if count > 1]

if duplicates:
    print("\n🚨 ZNALEZIONE DUPLIKATY ID:")
    for id_val, count in duplicates:
        print(f"\n  ID '{id_val}' występuje {count} razy:")
        
        # Pokaż szczegóły dla każdego wystąpienia
        problems_with_id = [p for p in data if p.get('id') == id_val]
        for i, p in enumerate(problems_with_id, 1):
            print(f"    Wersja {i}:")
            print(f"      - Statement: {p.get('statement', 'BRAK')[:80]}...")
            print(f"      - Difficulty: {p.get('difficulty')}")
            print(f"      - Steps: {len(p.get('steps', []))}")
            print(f"      - Solution: {p.get('solutions', ['BRAK'])[0] if p.get('solutions') else 'BRAK'}")
else:
    print("\n✅ Brak duplikatów ID")

## 11. Timestamp Analysis

In [ ]:
# Analiza znaczników czasu
timestamps = [p.get('timestamp') for p in data if 'timestamp' in p]

print(f"\n📅 Analiza Timestamp:")
print(f"  Zadania z timestamp: {len(timestamps)}/{len(data)}")

if timestamps:
    # Grupowanie po datach
    dates = [ts.split('T')[0] for ts in timestamps]
    date_counts = Counter(dates)
    
    print("\n  Rozkład edycji po datach:")
    for date in sorted(date_counts.keys()):
        print(f"    {date}: {date_counts[date]} zadań")
    
    # Wykres
    if len(date_counts) > 1:
        plt.figure(figsize=(12, 5))
        dates_sorted = sorted(date_counts.items())
        x = [d[0] for d in dates_sorted]
        y = [d[1] for d in dates_sorted]
        plt.bar(x, y, color='purple', alpha=0.7)
        plt.title('Liczba Edycji Zadań według Dat', fontsize=14, fontweight='bold')
        plt.xlabel('Data')
        plt.ylabel('Liczba zadań')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

## 12. Problematyczne Zadania

In [ ]:
# Identyfikacja potencjalnie problematycznych zadań
problematic = []

for p in data:
    issues = []
    
    # Sprawdzenie brakujących pól
    if not p.get('statement'):
        issues.append('brak statement')
    if not p.get('solutions'):
        issues.append('brak rozwiązania')
    if not p.get('steps'):
        issues.append('brak kroków')
    if '$' not in p.get('statement', ''):
        issues.append('brak LaTeX w statement')
    
    # Sprawdzenie validation
    val = p.get('validation', {})
    if not val.get('isValid', False):
        issues.append('validation: nie jest valid')
    if val.get('warnings'):
        issues.append(f"validation warnings: {len(val['warnings'])}")
    if val.get('missingFields'):
        issues.append(f"brakujące pola: {val['missingFields']}")
    
    # Bardzo mało lub bardzo dużo kroków
    num_steps = len(p.get('steps', []))
    if num_steps < 3:
        issues.append(f'tylko {num_steps} kroki')
    elif num_steps > 20:
        issues.append(f'aż {num_steps} kroków')
    
    if issues:
        problematic.append({
            'id': p.get('id'),
            'issues': issues
        })

if problematic:
    print("\n⚠️  Potencjalnie problematyczne zadania:")
    print(f"  Znaleziono: {len(problematic)} zadań\n")
    
    for item in problematic[:10]:  # Pokaż pierwsze 10
        print(f"  {item['id']}:")
        for issue in item['issues']:
            print(f"    - {issue}")
else:
    print("\n✅ Nie znaleziono problematycznych zadań")

## 13. Podsumowanie i Rekomendacje

In [ ]:
print("\n" + "="*70)
print("📋 PODSUMOWANIE ANALIZY")
print("="*70)

print(f"\n✅ MOCNE STRONY:")
print(f"  • Wszystkie zadania mają kompletne pola (hint, expression, explanation)")
print(f"  • Wysoka jakość interactive choices - wszystkie z feedbackiem")
print(f"  • Konsystentna struktura danych")
print(f"  • Dobre pokrycie różnych poziomów trudności")

print(f"\n⚠️  DO POPRAWY:")
if duplicates:
    print(f"  • Znaleziono {len(duplicates)} duplikaty ID - wymagają unikalnych identyfikatorów")
if validation_stats['has_warnings'] > 0:
    print(f"  • {validation_stats['has_warnings']} zadań ma ostrzeżenia walidacji")
if validation_stats['has_missing_fields'] > 0:
    print(f"  • {validation_stats['has_missing_fields']} zadań ma brakujące pola")

# Brak LaTeX
no_latex = sum(1 for p in data if '$' not in p.get('statement', ''))
if no_latex > 0:
    print(f"  • {no_latex} zadań nie ma formatowania LaTeX w statement")

print(f"\n📊 STATYSTYKI KOŃCOWE:")
print(f"  • Całkowita liczba zadań: {len(data)}")
print(f"  • Całkowita liczba kroków: {sum(len(p.get('steps', [])) for p in data)}")
print(f"  • Średnia kroków na zadanie: {sum(len(p.get('steps', [])) for p in data) / len(data):.1f}")
print(f"  • Całkowita liczba interactive choices: {sum(1 for p in data for s in p.get('steps', []) if 'interactive_choice' in s)}")
print(f"  • Zadania z validation: {validation_stats['has_validation']}/{len(data)}")
print(f"  • Zadania z rozwiązaniami: {solution_analysis['single_solution'] + solution_analysis['multiple_solutions']}/{len(data)}")